#### Problem Statement: Classifying if a customer is Member or Non-member

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import mlflow

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score
from sklearn.metrics import mutual_info_score
from sklearn.metrics import f1_score

ModuleNotFoundError: No module named 'mlflow'

In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("walmart-customer-experiment")

2024/03/19 13:00:51 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/03/19 13:00:51 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade 97727af70f4d -> 3500859a5d39, Add Model Aliases table
INFO  [alembic.runtime.migration] Running upgrade 3500859a5d39 -> 7f2a7d5fae7d, add datasets inputs input_tags tables
INFO  [alembic.runtime.migration] Running upgrade 7f2a7d5fae7d -> 2d6e25af4d3e, increase max param val length from 500 to 8000
INFO  [alembic.runtime.migration] Running upgrade 2d6e25af4d3e -> acf3f17fdcc7, add storage location field to model versions
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1710858642233, experiment_id='1', last_update_time=1710858642233, lifecycle_stage='active', name='walmart-customer-experiment', tags={}>

In [8]:
directory_path = 'D:/code/mlops-zoomcamps/01-intro/training/data'

os.chdir(directory_path)

files = os.listdir()
df = pd.DataFrame()

for file_name in files:
    if os.path.isfile(file_name):
        df = pd.read_csv(file_name, encoding='latin1',sep=';')

ParserError: Error tokenizing data. C error: Expected 2 fields in line 46, saw 3


In [32]:
df.head()

""


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame


In [17]:
df.columns

Index(['invoice_id', 'branch', 'city', 'customer_type', 'gender',
       'product_line', 'unit_price', 'quantity', 'vat', 'total', 'dtme', 'tme',
       'payment_method', 'cogs', 'gross_margin_pct', 'gross_income', 'rating',
       'time_of_day', 'day_name', 'month_name'],
      dtype='object')

In [5]:
df.shape

(1000, 20)

In [10]:
df.isnull().sum()

invoice_id          0
branch              0
city                0
customer_type       0
gender              0
product_line        0
unit_price          0
quantity            0
vat                 0
total               0
dtme                0
tme                 0
payment_method      0
cogs                0
gross_margin_pct    0
gross_income        0
rating              0
time_of_day         0
day_name            0
month_name          0
dtype: int64

In [18]:
numerical = df.select_dtypes(exclude='O').columns
df[numerical].columns

Index(['unit_price', 'quantity', 'vat', 'total', 'cogs', 'gross_margin_pct',
       'gross_income', 'rating'],
      dtype='object')

In [19]:
categorical = df.select_dtypes(include='O').columns
df[categorical].columns

Index(['invoice_id', 'branch', 'city', 'customer_type', 'gender',
       'product_line', 'dtme', 'tme', 'payment_method', 'time_of_day',
       'day_name', 'month_name'],
      dtype='object')

In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
unit_price,1000.0,55.672130,26.494628,10.0800,32.875000,55.2300,77.93500,99.9600
quantity,1000.0,5.510000,2.923431,1.0000,3.000000,5.0000,8.00000,10.0000
vat,1000.0,15.379369,11.708825,0.5085,5.924875,12.0880,22.44525,49.6500
total,1000.0,322.966749,245.885335,10.6785,124.422375,253.8480,471.35025,1042.6500
cogs,1000.0,307.587380,234.176510,10.1700,118.497500,241.7600,448.90500,993.0000
gross_margin_pct,1000.0,4.761900,0.000000,4.7619,4.761900,4.7619,4.76190,4.7619
gross_income,1000.0,15.379369,11.708825,0.5085,5.924875,12.0880,22.44525,49.6500
rating,1000.0,6.972700,1.718580,4.0000,5.500000,7.0000,8.50000,10.0000


In [22]:
df.customer_type.value_counts()

Member    501
Normal    499
Name: customer_type, dtype: int64

array(['Ewallet', 'Cash', 'Credit card'], dtype=object)